# **A Tour of the pyclep package's Ion class**

The primary class used for calculations with pyCELP is the Ion class.  It is automatically imported into the main package from the pyclep.ion sub-module.  With it, one can load the data for a single ion and perform calculations relevant to synthesizing the polarized emission.

Let's start by initializing a class for the Fe XIII ion.  We aim to explore the 1074.6 and 1079.8 nm coronal emission lines.  When you initialize the Ion class, pycelp will read the chianti database for the given atom.  The default to is read all level and collisional information available.  For the Fe XIII ion, there are 749 energy levels.  Once the data is read, pyclep will then pre-calculate as many terms (or factors) in the statistical equilbrium equations as possible.  This includes all calculates of the relevant wigner coefficients.  As such, the initialization can take some time.  

Here we will restrict the number of levels we use in our calculations to make it faster.  It is up to the user to decide how many levels are appropriate for a given atom.  See Schad & Dima (2020) for more details.  

In [68]:
import pycelp 
import numpy as np

In [69]:
fe13 = pycelp.Ion('fe_13',nlevels = 50)

 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


**NOTE**:  If this is your first time running pycelp, initization may take longer.  pycelp using numba @njit compilation with caching enabled.  Once the various functions are run one, the compiled functions will be in the memory for this session and available in the cache for later.  Thus, subsequent executions will be much faster. 

Now, we can view the representation of the Ion by simplying return the class instance name directly

In [70]:
fe13

pyCELP Ion class
    ---------------------
    Ion Name: fe_13
    Number of energy levels included: 50
    Number of SEE equations: 142
    Number of Radiative Transitions: 366
    Ionization Equilbrium Filename: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq

We see that there at 50 energy levels included.  The number of statistical equilbrium equations is 142; thus, the rate matrix will be 142 x 142 in size.  The number of radiative transitions included is 366. 

We can look at the first 10 radiative transitions using the show_lines method. 

In [71]:
fe13.show_lines(10)

 Index -- WV_VAC [A] -- WV_AIR [A] -- TRANSITION
0 10749.105 10746.153 3s2 3p2 3P1  -->  3s2 3p2 3P0
1 10800.77 10797.804 3s2 3p2 3P2  -->  3s2 3p2 3P1
2 5387.438 5385.939 3s2 3p2 3P2  -->  3s2 3p2 3P0
3 2579.54 2578.768 3s2 3p2 1D2  -->  3s2 3p2 3P1
4 3388.911 3387.939 3s2 3p2 1D2  -->  3s2 3p2 3P2
5 2080.313 2079.65 3s2 3p2 1D2  -->  3s2 3p2 3P0
6 1370.815 1370.815 3s2 3p2 1S0  -->  3s2 3p2 3P2
7 2301.957 2301.248 3s2 3p2 1S0  -->  3s2 3p2 1D2
8 1216.428 1216.428 3s2 3p2 1S0  -->  3s2 3p2 3P1
9 600.405 600.405 3s 3p3 5S2  -->  3s2 3p2 1D2


Most variables in the class are currently in public scope.  For example, the level information read from Chianti is in the dictionary elvl_data.  We can look at the available the keys here.  pycelp uses much of this information for its calculations but it does not typically use this dictionary directly.  Reading the Ion class source code can helpful if interested

In [72]:
print(' elvl_data keys:')
print(fe13.elvl_data.keys())
print(' ')
print(' Chianti level index  ---   energy [' + fe13.elvl_data['energy_units'] + '] --- level configuration') 
print(' ... printing for first 5 levels')
for lev in range(5):
    print(fe13.elvl_data['index'][lev],' ',fe13.elvl_data['energy'][lev],'  ',fe13.elvl_data['full_level'][lev])

 elvl_data keys:
dict_keys(['ion_name', 'ion_z', 'ion_n', 'filename', 'version', 'reference', 'index', 'conf', 'conf_latex', 'conf_index', 'term', 'term_latex', 'level', 'level_latex', 'full_level', 'full_level_latex', 'label', 'mult', 's', 'l', 'l_sym', 'j', 'j_str', 'parity', 'parity_str', 'weight', 'obs_energy', 'theory_energy', 'energy', 'energy_units'])
 
 Chianti level index  ---   energy [cm^-1] --- level configuration
 ... printing for first 5 levels
1   0.0    3s2 3p2 3P0
2   9303.1    3s2 3p2 3P1
3   18561.699    3s2 3p2 3P2
4   48069.699    3s2 3p2 1D2
5   91511.0    3s2 3p2 1S0


Similar dictionaries are available for the radiation transitions (wgfa_data), electron collisional data (scups_data), and proton collisional data (splups_data). 

Now, lets do a single calculation of the statistical equilbrium.  The default method for this (currently) is called calc, which excepts an electron density, a temperature, a height above the solar surface and a magnetic field inclination.  The code currently assumes the proton temperature equals the electron temperature.  The proton density is set to 0.85 times the electron density.

In [73]:
edens = 1e8                  ## cm^-3
etemp = fe13.get_maxtemp()   ## Kelvin 
ht = 0.1                     ## solar radius units above the photosphere 
thetab = np.rad2deg(0.)      ## radians 
fe13.calc(edens,etemp,ht, thetab, include_limbdark=True, include_protons=True)

Having executed the calc method, the statistical equilibrium equations have now been solved and the elements of the atomic density matrix are now available.  The total rate matrix in the spherical tensor representation is given in the see_matrix variables.  Each row of this matrix corresponds to a particular energy level and moment order K, which is recorded in the see_lev and see_k variables. 

In [74]:
print(fe13.see_matrix.shape,fe13.see_lev.shape,fe13.see_k.shape)

(142, 142) (142,) (142,)


One can view the spherical tensor matrix elements directly in the rho variable, which has a shape of (nlevels x the maximum order K). Q = 0 on account of the no coherence hypothesis.  Thus, rho has already been reshaped using the index variables above. Associated with the index variables is an array of weights used to normalize the total populations to unity.  You can check the solution by ensuring the total population is 1 as follows.  A method may be created to do this directly. 

In [78]:
print(' Shape of rho: ',fe13.rho.shape)
## here is where you can calculate the trace of the equations 
print(' sum of populations: ',  np.sum(fe13.rho[:,0]*fe13.weight[fe13.weight != 0]))

 Shape of rho:  (50, 11)
 sum of populations:  1.0000000000000002
